In [1]:
import pickle
import math
import numpy as np
from param import Parameters

In [2]:
# r0103 = pickle.load(open("./mldata/mlresult0103","rb"))
# v0103 = pickle.load(open("./mldata/v_mlresult0103","rb"))

# r0104 = pickle.load(open("./mldata/mlresult0104","rb"))
# v0104 = pickle.load(open("./mldata/v_mlresult0104","rb"))

# r0105 = pickle.load(open("./mldata/mlresult0105","rb"))
# v0105 = pickle.load(open("./mldata/v_mlresult0105","rb"))

# r0106 = pickle.load(open("./mldata/mlresult0106","rb"))
# v0106 = pickle.load(open("./mldata/v_mlresult0106","rb"))

# r0107 = pickle.load(open("./mldata/mlresult0107","rb"))
# v0107 = pickle.load(open("./mldata/v_mlresult0107","rb"))

In [15]:
def get_ignored(data):
    count = 0
    count2 = 0
    length = len(data)
    for r in data:
        if r.ignored is True:
            count += 1
        else:
            count2 += 1
    # return (length, count2, count, 1-(count/length))
    return 1 - count / length

In [16]:
def get_shared(data):
    count = 0
    count1 = 0
    length = len(data)
    for r in data:
        if r.shared is True:
            count += 1
        if r.ignored is False:
            count1 += 1
    # return (count, count/count1)
    return count / count1

In [5]:
def get_dratio(data):
    r_sum = 0
    r_num = 0
    for r in data:
        if r.ignored is False and r.shared is True:
            sharedis = r.share_distance[1]+r.share_distance[2]
            dis = Parameters.DistanceDict[r.PuHub][r.DoHub]
            ratio = (sharedis - dis)/dis
            r_sum += ratio
            r_num += 1
    return(r_sum/r_num)

In [6]:
def get_sharedelay(data):
    sum = 0
    num = 0
    for r in data:
        if r.ignored is False and r.shared is True:
            sum += (r.DoTime-r.EarliestDoTime)
            num += 1
    return(sum/num)
    
    

In [7]:
def get_waitdelay(data):
    sum = 0
    num = 0
    for r in data:
        if r.ignored is False:
            sum += (r.PuTime-r.RequestTime)
            num += 1
    return(sum/num)

In [22]:
def get_mean_distance(veh):
    count6 = 0
    count8 = 0
    
    for v in veh:
        count6 += v.TravelTime
        count8 += v.IdleTotalTime
    return (count6-count8) * 6 / len(veh)

In [23]:
def get_mean_rb_distance(veh):
    count = 0
    for v in veh:
        count += v.RbTotalTime
    return count * 6 / len(veh)

In [24]:
def get_totalprice(data):
    count = 0
    for r in data:
        count += r.price
    return count

In [11]:
def get_stats(data):
    count = 0
    price = 0
    profit = 0
    cost = 0
    discount = 0
    pro = 0
    for r in data:
        count += 1
        discount += r.discount
        cost += r.estcost
        price += r.price
        profit += r.profit
        pro += r.probability

    print(price/count,cost/count,profit/count,discount/count,pro/count)

In [12]:
def get_error(data):
    error_count = 0
    count = 0
    for r in data:
        if r.ignored is False:
            cost = (r.share_distance[1]+r.share_distance[2]*0.5)*2.0587/1000
            error = abs(r.estcost-cost)/cost
            error_count += error
            count += 1
    print(error_count/count)

In [21]:
import pandas as pd 

In [31]:
# time window changes
header = {'MeanServiceRate': [], 'SharedRate': [], 'DetourRatio': [], 
          'Delay': [], 'Wait': [], 'MeanDist': [], 'MeanRbDist': [], 'GMV': []}
index = [10, 20, 30, 40, 50, 60]
for win in index:
    rres = pickle.load(open(f"./trainingdata2/r_0109_2000_300_600_{win}", 'rb'))
    vres = pickle.load(open(f"./trainingdata2/v_0109_2000_300_600_{win}", 'rb'))
    header['MeanServiceRate'].append(get_ignored(rres))
    header['SharedRate'].append(get_shared(rres))
    header['DetourRatio'].append(get_dratio(rres))
    header['Delay'].append(get_sharedelay(rres))
    header['Wait'].append(get_waitdelay(rres))
    header['MeanDist'].append(get_mean_distance(vres))
    header['MeanRbDist'].append(get_mean_rb_distance(vres))
    header['GMV'].append(get_totalprice(rres))

df = pd.DataFrame(header, index=index)
df.to_csv('0109_2000_300_600_windows.csv')
df

In [35]:
# date change
header = {'MeanServiceRate': [], 'SharedRate': [], 'DetourRatio': [], 
          'Delay': [], 'Wait': [], 'MeanDist': [], 'MeanRbDist': [], 'GMV': []}
index = ['20160103', '20160104', '20160105', '20160106', '20160107', '20160108', '20160109']
index2 = ['0103', '0104', '0105', '0106', '0107', '0108', '0109']
for date in index2:
    rres = pickle.load(open(f"./trainingdata2/r_{date}_2000_300_600_30", 'rb'))
    vres = pickle.load(open(f"./trainingdata2/v_{date}_2000_300_600_30", 'rb'))
    header['MeanServiceRate'].append(get_ignored(rres))
    header['SharedRate'].append(get_shared(rres))
    header['DetourRatio'].append(get_dratio(rres))
    header['Delay'].append(get_sharedelay(rres))
    header['Wait'].append(get_waitdelay(rres))
    header['MeanDist'].append(get_mean_distance(vres))
    header['MeanRbDist'].append(get_mean_rb_distance(vres))
    header['GMV'].append(get_totalprice(rres))
df = pd.DataFrame(header, index=index)
df.to_csv('dates_2000_300_600_30.csv')
df

,MeanServiceRate,SharedRate,DetourRatio,Delay,Wait,MeanDist,MeanRbDist,GMV
20160103,0.800953,0.937234,0.095794,184.011212,126.112757,20353.141660,3121.687160,185154.119237
20160104,0.988246,0.726073,0.070945,152.739416,85.779027,9116.161407,308.774262,67598.179334
20160105,0.994248,0.760125,0.089411,165.698415,94.146760,10105.336027,226.732118,76627.952569
20160106,0.995050,0.788149,0.084291,160.532910,94.250862,10698.069734,174.951498,80546.758095
20160107,0.996209,0.834982,0.105340,187.632733,114.237099,12017.372216,91.141248,90681.116663
20160108,0.968058,0.888721,0.107217,192.520330,126.057584,15753.543221,1053.797848,123110.522847
20160109,0.693062,0.947483,0.096173,195.124382,131.730362,21459.469102,2851.221626,201602.820016


In [39]:
# vehicle change
header = {'MeanServiceRate': [], 'SharedRate': [], 'DetourRatio': [], 
          'Delay': [], 'Wait': [], 'MeanDist': [], 'MeanRbDist': [], 'GMV': []}
index = [1000, 1500, 2000, 2500, 3000]
for veh in index:
    rres = pickle.load(open(f"./trainingdata2/r_0109_{veh}_300_600_30_120", 'rb'))
    vres = pickle.load(open(f"./trainingdata2/v_0109_{veh}_300_600_30_120", 'rb'))
    header['MeanServiceRate'].append(get_ignored(rres))
    header['SharedRate'].append(get_shared(rres))
    header['DetourRatio'].append(get_dratio(rres))
    header['Delay'].append(get_sharedelay(rres))
    header['Wait'].append(get_waitdelay(rres))
    header['MeanDist'].append(get_mean_distance(vres))
    header['MeanRbDist'].append(get_mean_rb_distance(vres))
    header['GMV'].append(get_totalprice(rres))
df = pd.DataFrame(header, index=index)
df.to_csv('0109_vehicles_300_600_30_120.csv')
df

,MeanServiceRate,SharedRate,DetourRatio,Delay,Wait,MeanDist,MeanRbDist,GMV
1000,0.361800,0.976666,0.138664,250.998817,145.163778,24873.917415,1795.593034,153949.396673
1500,0.545920,0.961384,0.117426,222.661546,138.703468,23250.651955,2264.530281,188560.560790
2000,0.695491,0.945341,0.096914,193.752536,130.676231,21378.810737,3009.378849,202590.600784
2500,0.855658,0.934132,0.092801,173.695421,118.499986,4595.834707,362.188687,216726.524230
3000,0.939001,0.913107,0.091329,160.871941,107.269237,17315.392733,2325.408236,220213.390881


In [40]:
# with or without reblance
header = {'MeanServiceRate': [], 'SharedRate': [], 'DetourRatio': [], 
          'Delay': [], 'Wait': [], 'MeanDist': [], 'MeanRbDist': [], 'GMV': []}
index = ['Reblance', 'No-Reblance']
index2 = ['', '_norb']
for rb in index2:
    rres = pickle.load(open(f"./trainingdata2/r_0109_2000_300_600_30{rb}", 'rb'))
    vres = pickle.load(open(f"./trainingdata2/v_0109_2000_300_600_30{rb}", 'rb'))
    header['MeanServiceRate'].append(get_ignored(rres))
    header['SharedRate'].append(get_shared(rres))
    header['DetourRatio'].append(get_dratio(rres))
    header['Delay'].append(get_sharedelay(rres))
    header['Wait'].append(get_waitdelay(rres))
    header['MeanDist'].append(get_mean_distance(vres))
    header['MeanRbDist'].append(get_mean_rb_distance(vres))
    header['GMV'].append(get_totalprice(rres))
df = pd.DataFrame(header, index=index)
df.to_csv('0109_2000_300_600_30_rb.csv')
df

,MeanServiceRate,SharedRate,DetourRatio,Delay,Wait,MeanDist,MeanRbDist,GMV
Reblance,0.693062,0.947483,0.096173,195.124382,131.730362,21459.469102,2851.221626,201602.820016
No-Reblance,0.641048,0.945650,0.102500,201.929130,130.513238,19886.374928,0.000000,197880.152203


In [25]:
r0109 = pickle.load(open("./trainingdata2/r_0109_2000_300_600_30", 'rb'))
v0109 = pickle.load(open("./trainingdata2/v_0109_2000_300_600_30", 'rb'))

In [26]:
print("Mean service rate:", get_ignored(r0109))
print("Percentage of shared rides:", get_shared(r0109))
print("Detour ratio:", get_dratio(r0109))
print("Mean in-car travel delay:", get_sharedelay(r0109))
print("Mean waiting time:", get_waitdelay(r0109))
print("Mean distance travelled:", get_mean_distance(v0109))
print("Total reblance distance:", get_mean_rb_distance(v0109))
print("Total GMV:", get_totalprice(r0109))
# print(get_totalprice(r0109) - (dis/1000*1.087+dis/6*0.00583)*0.75)
# get_stats(r0109)
# get_error(r0109)

Mean service rate: 0.6930620608899297
Percentage of shared rides: 0.9474832805350228
Detour ratio: 0.09617317140114526
Mean in-car travel delay: 195.12438187582015
Mean waiting time: 131.7303618561535
Mean distance travelled: 21459.46910249999
Total reblance distance: 2851.221625500001
Total GMV: 201602.82001575525


In [17]:
# r0109_3 = pickle.load(open("./trainingdata2/0109_300_300_600_120", 'rb'))
# v0109_3 = pickle.load(open("./trainingdata2/v_0109_300_300_600_120", 'rb'))

# print(get_ignored(r0109_3))
# print(get_shared(r0109_3))
# print(get_dratio(r0109_3))
# print(get_sharedelay(r0109_3))
# print(get_waitdelay(r0109_3))
# dis = get_totaldistance(v0109_3)
# print(dis)
# print(get_rbdistance(v0109_3))
# price = get_totalprice(r0109_3)
# print(price)
# print(price - (dis/1000*1.087+dis/6*0.00583)*0.75)
# # get_stats(r0109)
# # get_error(r0109)


(20492, 2065, 18427, 0.10077103259808706)
(2033, 0.9845036319612591)
0.13942623976203689
261.1122661091982
132.8389272800644
7971586.967999996
441552.521
65395.173021215895
53087.042742623904


In [16]:
# print(get_ignored(r0103))
# print(get_shared(r0103))
# print(get_dratio(r0103))
# print(get_sharedelay(r0103))
# print(get_waitdelay(r0103))
# dis = get_totaldistance(v0103)
# print(dis)
# print(get_rbdistance(v0103))
# price = get_totalprice(r0103)
# print(price)
# print(price - (dis/1000*1.087+dis/6*0.00583)*0.75)
# get_stats(r0103)
# get_error(r0103)


(292605, 283463, 9142, 0.9687565147553869)
(264144, 0.9318464843736219)
0.10685901198452478
183.12978647315708
126.00111022072991
773359991.341003
98778887.38000032
3206510.7581710857
2012442.9315405772
10.95849612334405 4.216848028318415 3.7814957768940096 0.7823160695056944 0.8578696928239937
0.23716617081010605


In [19]:
# print(get_ignored(r0104))
# print(get_shared(r0104))
# print(get_dratio(r0104))
# print(get_sharedelay(r0104))
# print(get_waitdelay(r0104))
# dis = get_totaldistance(v0104)
# print(dis)
# print(get_rbdistance(v0104))
# price = get_totalprice(r0104)
# print(price)
# print(price - (dis/1000*1.087+dis/6*0.00583)*0.75)
# get_stats(r0104)
# get_error(r0104)

(304191, 294209, 9982, 0.967185090946149)
(272785, 0.927181017575941)
0.11456120603496682
181.8087858062814
125.98719979617375
746025261.4300022
92901857.68699966
3180553.614064822
2028690.6104168987
10.45577815933023 3.8104795875271296 3.770692773316038 0.7763296030091845 0.8619896785780988
0.24704637968103457


In [20]:
# print(get_ignored(r0105))
# print(get_shared(r0105))
# print(get_dratio(r0105))
# print(get_sharedelay(r0105))
# print(get_waitdelay(r0105))
# dis = get_totaldistance(v0105)
# print(dis)
# print(get_rbdistance(v0105))
# price = get_totalprice(r0105)
# print(price)
# print(price - (dis/1000*1.087+dis/6*0.00583)*0.75)
# get_stats(r0105)
# get_error(r0105)

(330108, 318199, 11909, 0.963923927926618)
(297011, 0.9334127385692601)
0.11448515423032954
180.60191984640562
126.12977134382712
790270350.167999
94192170.72699988
3395360.2313886248
2175182.8107292345
10.285604200409033 3.7107967509721216 3.7341271198901453 0.7746885470053692 0.8628410024104756
0.2452668250032039


In [22]:
# print(get_ignored(r0106))
# print(get_shared(r0106))
# print(get_dratio(r0106))
# print(get_sharedelay(r0106))
# print(get_waitdelay(r0106))
# dis = get_totaldistance(v0106)
# print(dis)
# print(get_rbdistance(v0106))
# price = get_totalprice(r0106)
# print(price)
# print(price - (dis/1000*1.087+dis/6*0.00583)*0.75)
# get_stats(r0106)
# get_error(r0106)

(336172, 323624, 12548, 0.962673869328796)
(302884, 0.935913282080439)
0.11194236456881448
180.6461607287534
126.20372132165612
807732066.2390008
92445560.34600024
3461513.26149095
2214374.9512179326
10.29685179459012 3.745044163118359 3.7148516141973995 0.7758757933993901 0.862243382873158
0.24156022248417033


In [23]:
# print(get_ignored(r0107))
# print(get_shared(r0107))
# print(get_dratio(r0107))
# print(get_sharedelay(r0107))
# print(get_waitdelay(r0107))
# dis = get_totaldistance(v0107)
# print(dis)
# print(get_rbdistance(v0107))
# price = get_totalprice(r0107)
# print(price)
# print(price - (dis/1000*1.087+dis/6*0.00583)*0.75)
# get_stats(r0107)
# get_error(r0107)

(352513, 333424, 19089, 0.9458488055759646)
(314530, 0.9433334133115793)
0.11171062849710547
182.55132705783546
128.02827895862498
847417212.3020035
99361491.48200028
3647818.5873175133
2339406.41152322
10.348039894464923 3.807864160848784 3.7020921171487586 0.7767252960382489 0.8613251001861741
0.23894802710804977


In [36]:
# print(get_ignored(r0104))
# print(get_shared(r0104))
# print(get_dratio(r0104))
# print(get_sharedelay(r0104))
# print(get_waitdelay(r0104))
# dis = get_totaldistance(v0104)
# print(dis)
# print(get_rbdistance(v0104))
# price = get_totalprice(r0104)
# print(price)
# print(price - (dis/1000*1.087+dis/6*0.00583)*0.75)

(304191, 287735, 16456, 0.9459024099989809)
(271139, 0.9423219281630667)
0.12339891939834839
191.75397239054504
133.81337038594089
740800354.7229993
95080328.97300014
3175606.0544510814
2031810.3067587705


In [37]:
# print(get_ignored(r0103))
# print(get_shared(r0103))
# print(get_dratio(r0103))
# print(get_sharedelay(r0103))
# print(get_waitdelay(r0103))
# dis = get_totaldistance(v0103)
# print(dis)
# print(get_rbdistance(v0103))
# price = get_totalprice(r0103)
# print(price)
# print(price - (dis/1000*1.087+dis/6*0.00583)*0.75)

(292605, 281237, 11368, 0.9611489892517217)
(265735, 0.9448792299732965)
0.11255452129037744
190.9572063013851
133.09761673605695
771350210.2799962
101205626.57500018
3204228.5220575123
2013263.797385198
